<a href="https://colab.research.google.com/github/alberwan/spark-nlp-workshop/blob/master/java/annotation/JavaNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.5.1

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 63kB/s 
     |████████████████████████████████| 204kB 2.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=1812f9cc2dc8d4b91e6bfc54b1f22c36f3316e7131cfc4424bd94c985fff2ce3
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 122kB 2.9MB/s 


In [6]:
#package EDG.SparkNLP.com;

import com.johnsnowlabs.nlp.DocumentAssembler;
import com.johnsnowlabs.nlp.EmbeddingsFinisher;
import com.johnsnowlabs.nlp.LightPipeline;
import com.johnsnowlabs.nlp.annotators.LemmatizerModel;
import com.johnsnowlabs.nlp.annotators.Tokenizer;
#import com.johnsnowlabs.nlp.embeddings.*; 
import com.johnsnowlabs.nlp.embeddings.EmbeddingsHelper;
#import com.johnsnowlabs.nlp.embeddings.ClusterWordEmbeddings;
import com.johnsnowlabs.nlp.SparkNLP;
import com.johnsnowlabs.nlp.pretrained.PretrainedPipeline;
import org.apache.spark.ml.Pipeline;
import org.apache.spark.ml.PipelineModel;
import org.apache.spark.ml.PipelineStage;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.Encoders;
import org.apache.spark.sql.Row;
import org.apache.spark.sql.SparkSession;

import java.util.LinkedList;

public class SparkNLPAnnotation {

	public static void main(String[] args) {
		// TODO Auto-generated method stub
		DocumentAssembler document = new DocumentAssembler();
        document.setInputCol("text");
        document.setOutputCol("document");
        document.setCleanupMode("disabled");

        Tokenizer tokenizer = new Tokenizer();
        tokenizer.setInputCols(new String[] {"document"});
        tokenizer.setOutputCol("token");

        Pipeline pipeline = new Pipeline();
        pipeline.setStages(new PipelineStage[] {document, tokenizer});

        SparkSession spark = com.johnsnowlabs.nlp.SparkNLP.start(false, false);

        LinkedList<String> text = new java.util.LinkedList<String>();

        text.add("Peter is a very good person");

        Dataset<Row> data = spark.createDataset(text, Encoders.STRING()).toDF("text");

        PipelineModel pipelineModel = pipeline.fit(data);

        Dataset<Row> transformed = pipelineModel.transform(data);
        transformed.show();

        /*
         * Do Spark-NLP pretrained pipelines only work on linux systems?
         * https://stackoverflow.com/questions/57610129/do-spark-nlp-pretrained-pipelines-only-work-on-linux-systems
         * 
         */
        PretrainedPipeline pretrained = new PretrainedPipeline("explain_document_dl");
        //PretrainedPipeline pretrained = new PretrainedPipeline("explain_document_dl_noncontrib");
        pretrained.transform(data).show();

        LemmatizerModel lemmatizer = (LemmatizerModel) LemmatizerModel.pretrained("lemma_antbnc");
        lemmatizer.setInputCols(new String[] {"token"});
        lemmatizer.setOutputCol("lemma");

        lemmatizer.transform(transformed).show();

        LightPipeline lightPipeline = new LightPipeline(pipelineModel, true);

        java.util.Map<String, java.util.List<String>> result = lightPipeline.annotateJava("Peter is a very good person.");

        System.out.println(result.get("token"));

        java.util.ArrayList<String> list = new java.util.ArrayList<String>();
        list.add("Peter is a good person.");
        list.add("Roy lives in Germany.");

        System.out.println(lightPipeline.annotateJava(list));
        
        //EmbeddingsHelper.load("./random_embeddings_dim4.txt",spark,"TEXT","random",4,false);

        System.out.println("\nFinished testing Spark NLP on JAVA");
	}
}

SyntaxError: ignored